# 🧠 LangSmith 简介

---

## 🌐 什么是 LangSmith？

**LangSmith** 是 LangChain 官方提供的「可观测性与调试平台」。  
它能帮你记录和分析所有 LangChain / LangGraph 的执行过程，包括：

- ✅ 模型输入 / 输出  
- ✅ 每个链 (Chain) 或节点 (Node) 的执行顺序  
- ✅ Token 使用量与耗时  
- ✅ 子链 / 工具调用的嵌套层级  
- ✅ Prompt 实验和效果对比  

换句话说：
> 🧩 **LangChain 负责“编排”，LangSmith 负责“观察”。**

---

## 📊 LangSmith 的作用位置

它位于 LangChain 应用执行的最外层，用于监控所有运行。




---

## 💡 为什么要用 LangSmith？

| 场景 | 好处 |
|------|------|
| 调试复杂链 / Graph | 可视化执行轨迹 |
| Prompt 调优 | 快速比较不同版本输出 |
| 记录模型表现 | 自动保存输入 / 输出 |
| 成本分析 | 查看 token / 耗时 / 费用 |
| 团队协作 | 所有人可共享运行日志 |

---


# 接入 LangSmith

# 1. 环境变量配置

In [ ]:
import os
from getpass import getpass

# 打开 LangSmith 跟踪功能
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "xxx"
os.environ["LANGCHAIN_PROJECT"] = "LangGraph_Demo"  # 可自定义项目名

print("LangSmith 环境变量已配置完成！")

LangSmith 环境变量已配置完成！


# 2. 测试是否连接成功

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 定义一个简单的 LCEL 链
llm = ChatOpenAI(
    model="gpt-4o-mini",
    base_url="https://www.dmxapi.cn/v1",
    #api_key="",  替换为你的 API Key 
    temperature=0
)

qa_chain = (
    ChatPromptTemplate.from_template("请简要回答：{question}")
    | llm
    | StrOutputParser()
)

# 调用一次
result = qa_chain.invoke({"question": "什么是 LangSmith？"})
print("模型回答：", result)

print("\n 调用已完成！现在可登录 https://smith.langchain.com 查看运行记录。")


模型回答： LangSmith 是一个专注于自然语言处理和人工智能的工具或平台，旨在帮助开发者和企业更好地构建、管理和优化与语言相关的应用程序。它可能提供模型训练、数据处理、API 接口等功能，以支持各种语言任务，如文本生成、情感分析和对话系统等。具体功能和应用场景可能会根据其最新版本和更新而有所不同。

 调用已完成！现在可登录 https://smith.langchain.com 查看运行记录。


# 使用 LangSmith Trace 装饰器追踪自定义函数
https://docs.langchain.com/langsmith/observability-quickstart

In [ ]:
# =====================================================
# LangSmith Trace 案例：最小 RAG 追踪
# =====================================================

from openai import OpenAI
from langsmith.wrappers import wrap_openai
from langsmith import traceable

# 1.模拟一个简单的检索器
def retriever(query: str):
    """模拟检索结果"""
    return ["Harrison worked at Kensho"]

# 2.使用 LangSmith 包装 OpenAI 客户端
# wrap_openai() 会自动捕获模型输入/输出并发送到 LangSmith
client = wrap_openai(OpenAI(
    base_url="https://www.dmxapi.cn/v1",
    #api_key="",  替换为你的 API Key 
))

# 3.用 traceable 装饰函数，让 LangSmith 记录整个 RAG 流程
@traceable
def rag(question: str) -> str:
    """一个最小的检索增强问答函数"""
    docs = retriever(question)
    system_message = (
        "Answer the user's question using only the provided information below:\n"
        + "\n".join(docs)
    )
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ],
    )
    return resp.choices[0].message.content


# 4.调用函数并打印结果
if __name__ == "__main__":
    print("模型回答：", rag("什么是AI"))


模型回答： AI（人工智能）指的是使计算机或计算系统能够执行通常需要人类智能的任务的技术。这些任务包括学习、推理、问题解决、理解自然语言、感知等。
